In [6]:
# Define dependances
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import urllib.request
import json 
from scipy.stats import linregress
from urllib.request import Request, urlopen
from pprint import pprint

# Import the API key
from api_keys import api_key

In [7]:
# Defining data
url = "https://api.cyclestreets.net/v2/collisions.locations?"

# Define coordinates
w = -0.19529421925045196
s = 51.4715990553213
e = -0.06041884946064573
n = 51.54115686360122
bbox = f"{w},{s},{e},{n}"

query_url = f"{url}key={api_key}&bbox={bbox}&casualtiesinclude=cyclist&limit=3&datetime=friendly&jitter=1&zoom=17"
response = requests.get(query_url).json()
# pprint(response)

In [8]:
# Convert dictionary to dataframe
s1 = json.dumps(response)
d1 = json.loads(s1)
df = pd.json_normalize(d1['features'])

In [9]:
# Print length of dataframe
len(df)

28506

In [10]:
# Drop unneeded columns
cycledf = df.drop(columns=["properties.url","properties.apiUrl", "geometry.type"])

#rename columns
cycle_df = cycledf.rename(columns={"properties.id": "ID",
          "properties.datetime" : "Date&Time",
          "properties.severity" : "Severity",
          "properties.casualties" : "Casualties",
          "properties.number_of_casualties" : "# of Casualties",
          "properties.number_of_vehicles" : "# of Vehicles",
          "properties.latitude" : "Latitude",
          "properties.longitude" : "Longitude",
          "geometry.coordinates" : "Coordinates"})

# Display changes
# cycle_df.head()

In [11]:
# Printing out csv file
cycle_df.to_csv("output_data/cyclingstreets_central_london.csv")

In [12]:
# Read csv file
cycle_df = pd.read_csv("output_data/cyclingstreets_central_london.csv")

# Display data
# cycle_df.head()

In [13]:
# Convert string columns to float
converted_cycle_df = cycle_df[["# of Casualties","# of Vehicles","Latitude","Longitude"]].astype(float)

In [16]:
%%capture --no-display
import hvplot.pandas

# Configure the map plot
map1 = converted_cycle_df.hvplot.points("Longitude","Latitude", size=20, geo=True, tiles="OSM", color="red",
                             frame_width = 800, frame_height = 600, scale = 0.5)

# Save the figure
hvplot.save(map1, 'centrallondoncycle.html')

# Cannot seem to make this code work even with installing "Selenium"
# hvplot.save(map1, 'centrallondoncycle.png')

# Display the map
map1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]